In [ ]:
import os
import sys
import json
import itertools

import json
import matplotlib.pyplot as plt
import numpy as np


%matplotlib inline

from IPython.display import set_matplotlib_formats
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
#script to process train/valiation results 
data_root = "results/rl4im/sacred"
runs = [1, 2, 3, 4, 5]
colge_im_graph_info = {}
colge_im_config = {}
colge_im_raw = {}
colge_im = {}


for r in runs:
    with open(os.path.join(data_root, str(r), 'graph_info.json')) as f:
        colge_im_graph_info[r] = json.load(f)

for r in runs:
    with open(os.path.join(data_root, str(r), 'eval_episode_rewards.json')) as f:
        colge_im_raw[r] = json.load(f)

for r in runs:
    with open(os.path.join(data_root, str(r), 'config.json')) as f:
        colge_im_config[r] = json.load(f)

for r in runs:
    key = list(colge_im_raw[r].keys())[0]
    #print(np.array(colge_im_raw[r][key]).shape)


for r in runs:
    key = list(colge_im_raw[r].keys())
    #print(key)
    

for r in runs:
    colge_im[r] = {}
    graphs = list(colge_im_raw[r].keys())
    
    for graph in graphs:
        colge_im[r][graph] = {}
        colge_im[r][graph]['mean_std'] = (np.mean(colge_im_raw[r][graph], axis=1), np.std(colge_im_raw[r][graph], axis=1))
        colge_im[r][graph]['normalized'] = np.array(colge_im_raw[r][graph]) 
        
        cap_from = int(np.ceil(500 / colge_im_config[r]['save_every']))
        cap_to = int(np.ceil(2000 / colge_im_config[r]['save_every']))
        colge_im[r][graph]['normalized_stable_epsilon'] = colge_im[r][graph]['normalized'][cap_from:cap_to, :]
        
        
    for graph in graphs:
        colge_im[r]['cat_norm'] = np.mean(np.concatenate([colge_im[r][graph]['normalized_stable_epsilon'] for graph in graphs], axis=1), axis=1)
    opt_id = np.argmax(colge_im[r]['cat_norm'])
    opt_checkpoint = (opt_id +1 + cap_from)*colge_im_config[r]['save_every']
    opt_checkpoint = opt_checkpoint + (colge_im_config[r]['T']-opt_checkpoint%colge_im_config[r]['T'])%colge_im_config[r]['T']
    max_value = np.max(colge_im[r]['cat_norm'])
    print('r:', r)
    print('capped normalized influence vector size is: ', colge_im[r]['cat_norm'].shape)
    print('opt id is:', opt_id)
    print('opt_checkpoint is: ', opt_checkpoint)
    print('max value is: ', max_value)
    print()